In [1]:
import numpy as np
import pandas as pd
digit = pd.read_csv('MNIST.csv')
digit.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [2]:
digit.shape

(42000, 785)

In [3]:
X = digit.iloc[:, 1:].values.reshape(len(digit), 28, 28, 1)
y = digit.iloc[:, 0].values

In [4]:
y

array([1, 0, 1, ..., 7, 6, 9])

In [5]:
#Converting target to categorical features
from keras.utils.np_utils import to_categorical
y = to_categorical(y)

In [6]:
y  #categorical formate

array([[ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.]])

In [7]:
print(X.shape, y.shape)

(42000, 28, 28, 1) (42000, 10)


In [8]:
#split the data into train and test 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=7)

#### CNN model

In [10]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D

In [11]:
model = Sequential()
model.add(Convolution2D(32, 3, 3, input_shape=(28, 28, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(100))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

/home/surya/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(28, 28, 1..., activation="relu")`
  


In [12]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 5408)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               540900    
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1010      
Total params: 542,230
Trainable params: 542,230
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=12, batch_size=128)

Train on 33600 samples, validate on 8400 samples
Epoch 1/12
33600/33600 [==============================] - 27s 818us/step - loss: 10.1980 - acc: 0.3635 - val_loss: 7.3631 - val_acc: 0.5406
Epoch 2/12
33600/33600 [==============================] - 28s 833us/step - loss: 4.5227 - acc: 0.7132 - val_loss: 2.6743 - val_acc: 0.8301
Epoch 3/12
33600/33600 [==============================] - 31s 932us/step - loss: 2.6765 - acc: 0.8292 - val_loss: 2.3081 - val_acc: 0.8533
Epoch 4/12
33600/33600 [==============================] - 32s 952us/step - loss: 2.4320 - acc: 0.8447 - val_loss: 1.6341 - val_acc: 0.8920
Epoch 5/12
33600/33600 [==============================] - 32s 959us/step - loss: 0.9966 - acc: 0.9322 - val_loss: 0.6308 - val_acc: 0.9574
Epoch 6/12
33600/33600 [==============================] - 35s 1ms/step - loss: 0.8203 - acc: 0.9443 - val_loss: 0.5670 - val_acc: 0.9614
Epoch 7/12
33600/33600 [==============================] - 34s 1ms/step - loss: 0.7202 - acc: 0.9516 - val_loss: 0.5822

In [15]:
#Predicting test data
y_pred = model.predict(X_test)

In [16]:
y_pred

array([[  0.00000000e+00,   1.00000000e+00,   0.00000000e+00, ...,
          0.00000000e+00,   5.28500460e-33,   0.00000000e+00],
       [  0.00000000e+00,   1.00000000e+00,   0.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       ..., 
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   1.00000000e+00,   0.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   1.00000000e+00,   0.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00]], dtype=float32)

In [17]:
#Arrange
y_test_cls = np.argmax(y_test, axis=1)
y_pred_cls = np.argmax(y_pred, axis=1)

In [18]:
y_pred_cls

array([1, 1, 4, ..., 4, 1, 1])

In [20]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test_cls, y_pred_cls)

In [21]:
cm

array([[828,   1,   0,   0,   0,   0,   3,   0,   1,   0],
       [  0, 945,   4,   3,   1,   0,   1,   3,   0,   1],
       [  5,   3, 784,   3,   8,   0,   1,   7,   4,   0],
       [  4,   0,  12, 844,   1,   6,   0,   3,   5,   4],
       [  1,   4,   0,   0, 801,   0,   5,   2,   0,   7],
       [  7,   1,   0,   6,   0, 699,   9,   0,   5,   5],
       [  6,   1,   0,   0,   3,   1, 780,   0,   3,   0],
       [  3,   2,   8,   3,   4,   1,   0, 831,   1,  30],
       [ 16,  16,   3,   1,   4,   2,   5,   1, 782,  11],
       [  5,   2,   1,   3,   8,   1,   0,   8,   1, 816]])

In [22]:
#For accuracy 
accuracy_score(y_test_cls, y_pred_cls)*100

96.547619047619051

In [23]:
# Evaluate data
score = model.evaluate(X_test, y_test)
print('Accuracy : ', score[1]*100)

8400/8400 [==============================] - 3s 324us/step
Accuracy :  96.5476190476
